# Recomendação de habilidades técnicas

In [1]:
import networkx as nx

In [2]:
# Importando o grafo completo
G = nx.read_gexf("job_skill_graph.gexf")

In [3]:
# Analisando os tipos de nós
node_types = nx.get_node_attributes(G, "type")
print("Tipos de nós no grafo:", set(node_types.values()))

Tipos de nós no grafo: {'job', 'skill'}


In [4]:
# Separando conhecimentos técnicos e vagas de emprego no grafo
S = [n for n, d in G.nodes(data=True) if d["type"] == "skill"]
J = [n for n, d in G.nodes(data=True) if d["type"] == "job"]

In [5]:
# Representando a rede por uma matriz M de adjacência entre vagas e conhecimentos técnicos
M = nx.bipartite.biadjacency_matrix(
    G, row_order=J, column_order=S, weight=None
).toarray()

## Algoritmo principal

In [6]:
def recommend_next_skill(C, J, S):
    """
    Recommends the next skill to acquire based on current skills and job requirements.
    Args:
        C (list): A binary list indicating known (1) and unknown (0) skills.
        J (list): List of job identifiers.
        S (list): List of skill identifiers.
    Returns:
        str or None: The recommended next skill to acquire, or None
    """
    T_J = []  # total de conhecimentos técnicos necessários para cada vaga de emprego
    C_J = []  # conhecimentos técnicos já conhecidos para cada vaga de emprego
    for job_index in range(len(J)):
        T_Ji = 0
        C_Ji = 0
        for skill_index in range(len(S)):
            if M[job_index][skill_index] == 1:
                T_Ji += 1
                if C[skill_index] == 1:
                    C_Ji += 1
        # Armazenando os resultados para uso futuro
        C_J.append(C_Ji)
        T_J.append(T_Ji)

    next_skill = None
    maximum_jobs = -1
    maximum_success = -1

    for skill_k_index, skill_k in enumerate(S):
        # Considerando apenas os conhecimentos técnicos ainda não adquiridos
        if C[skill_k_index] == 1:
            continue

        jobs_k = 0
        success_k = 0

        for job_index in range(len(J)):
            known_skills = C_J[job_index]
            if M[job_index][skill_k_index] == 1:
                known_skills += 1

            percentage_k = known_skills / T_J[job_index]
            if percentage_k >= 0.5:
                jobs_k += 1
                success_k += percentage_k

        if jobs_k > maximum_jobs:
            maximum_jobs = jobs_k
            maximum_success = success_k
            next_skill = skill_k
        elif jobs_k == maximum_jobs and success_k > maximum_success:
            maximum_success = success_k
            next_skill = skill_k

    return next_skill

## Testes de recomendação de próximos conhecimentos técnicos

In [7]:
# Criando C para uma determinada lista de conhecimentos técnicos já adquiridos
def create_C(S, known_skills):
    """
    Creates a binary list indicating known (1) and unknown (0) skills based on a given list of known skills.
    Args:
        S (list): List of skill identifiers.
        known_skills (list): List of known skill identifiers.
    Returns:
        list: A binary list where 1 indicates a known skill and 0 indicates an unknown skill.
    """
    C = []
    for skill in S:
        if skill in known_skills:
            C.append(1)
        else:
            C.append(0)
    return C

In [8]:
# Função auxiliar para avaliar o impacto do aprendizado de novos conhecimentos técnicos
def get_viable_jobs_count(C, J, S):
    """
    Counts the number of viable jobs (jobs for which the user has at least 50% of the required skills).
    Args:
        C (list): A binary list indicating known (1) and unknown (0) skills.
        J (list): List of job identifiers.
        S (list): List of skill identifiers.
    Returns:
        int: The number of viable jobs.
    """
    viable_jobs_count = 0
    for job_index in range(len(J)):
        known_skills = 0
        total_skills = 0

        for skill_index in range(len(S)):
            if M[job_index][skill_index] == 1:
                total_skills += 1
                if C[skill_index] == 1:
                    known_skills += 1

        if known_skills / total_skills >= 0.5:
            viable_jobs_count += 1
    return viable_jobs_count

### Teste 1: qual o primeiro conhecimento técnico a ser buscado?

In [9]:
# Sem conhecimentos técnicos prévios
first_skill_C = create_C(S, set())

first_skill = recommend_next_skill(first_skill_C, J, S)

before_learning = get_viable_jobs_count(first_skill_C, J, S)
print("Vagas de emprego viáveis antes de aprender:", before_learning)
print("Conhecimento técnico recomendado:", first_skill)

first_skill_C[S.index(first_skill)] = (
    1  # Atualizando C após aprender o primeiro conhecimento técnico recomendado
)
after_learning = get_viable_jobs_count(first_skill_C, J, S)
print(
    f"Vagas de emprego viáveis após aprender: {after_learning} (+{after_learning - before_learning})"
)

Vagas de emprego viáveis antes de aprender: 0
Conhecimento técnico recomendado: sap
Vagas de emprego viáveis após aprender: 1702 (+1702)


### Teste 2: qual conhecimento técnico buscar após as matérias iniciais da Unicamp?

In [10]:
# Catálogo de 2026 - MC102: Python; MC202: C; MC322: Java; MC536: SQL
initial_skills = {"python", "c", "java", "sql"}
initial_skills_C = create_C(S, initial_skills)

next_skill = recommend_next_skill(initial_skills_C, J, S)

before_learning = get_viable_jobs_count(initial_skills_C, J, S)
print("Vagas de emprego viáveis antes de aprender:", before_learning)
print("Conhecimento técnico recomendado:", next_skill)

initial_skills_C[S.index(next_skill)] = (
    1  # Atualizando C após aprender o conhecimento técnico recomendado
)
after_learning = get_viable_jobs_count(initial_skills_C, J, S)
print(
    f"Vagas de emprego viáveis após aprender: {after_learning} (+{after_learning - before_learning})"
)

Vagas de emprego viáveis antes de aprender: 4651
Conhecimento técnico recomendado: power bi
Vagas de emprego viáveis após aprender: 6925 (+2274)


### Teste 3: currículo focado em uma área

In [11]:
# Importando o grafo com as comunidades identificadas
skill_communities = nx.read_gexf("subgraphs/projection_skills_graph_simple_weight.gexf")

In [12]:
# Com os resultados da análise anterior
central_skills_per_community = {
    1: {"java", "git", "react", "javascript", "html"},
    2: {"ci/cd", "linux", "kubernetes", "docker", "go"},
    3: {"sql", "python", "aws", "azure", "gcp"},
    4: {"photoshop", "figma", "illustrator", "indesign", "sketch"},
    5: {"scrum", "kanban", "c#", "agile", "azure devops"},
}

for community_id, central_skills in central_skills_per_community.items():
    community_skills_C = create_C(S, central_skills)

    next_community_skill = recommend_next_skill(community_skills_C, J, S)

    print(f"Comunidade {community_id}:")
    before_learning = get_viable_jobs_count(community_skills_C, J, S)
    print("  Vagas de emprego viáveis antes de aprender:", before_learning)
    print("  Conhecimento técnico recomendado:", next_community_skill)

    community_skills_C[S.index(next_community_skill)] = (
        1  # Atualizando C após aprender o conhecimento técnico recomendado
    )
    after_learning = get_viable_jobs_count(community_skills_C, J, S)
    print(
        f"  Vagas de emprego viáveis após aprender: {after_learning} (+{after_learning - before_learning})"
    )

Comunidade 1:
  Vagas de emprego viáveis antes de aprender: 1720
  Conhecimento técnico recomendado: sap
  Vagas de emprego viáveis após aprender: 3478 (+1758)
Comunidade 2:
  Vagas de emprego viáveis antes de aprender: 2035
  Conhecimento técnico recomendado: sap
  Vagas de emprego viáveis após aprender: 3753 (+1718)
Comunidade 3:
  Vagas de emprego viáveis antes de aprender: 4628
  Conhecimento técnico recomendado: power bi
  Vagas de emprego viáveis após aprender: 7032 (+2404)
Comunidade 4:
  Vagas de emprego viáveis antes de aprender: 2657
  Conhecimento técnico recomendado: sap
  Vagas de emprego viáveis após aprender: 4360 (+1703)
Comunidade 5:
  Vagas de emprego viáveis antes de aprender: 2178
  Conhecimento técnico recomendado: sap
  Vagas de emprego viáveis após aprender: 3950 (+1772)


## Verificando o porquê dos resultados

In [13]:
skills_and_viable_jobs = {}
for job in J:
    total_skills = G.degree(job)
    # Se exige menos de 3 conhecimentos técnicos, é um job viável ao aprender apenas um requisito
    if total_skills < 3:
        job_skills = list(G.neighbors(job))
        for job_skill in job_skills:
            skills_and_viable_jobs[job_skill] = (
                skills_and_viable_jobs.get(job_skill, 0) + 1
            )

# Imprimindo os resultados dos 3 maiores
skills_and_viable_jobs_sorted = dict(
    sorted(skills_and_viable_jobs.items(), key=lambda item: item[1], reverse=True)
)
for skill, count in list(skills_and_viable_jobs_sorted.items())[:3]:
    print(f"Ao aprender apenas {skill}, há {count} vagas viáveis")

Ao aprender apenas sap, há 1702 vagas viáveis
Ao aprender apenas power bi, há 1508 vagas viáveis
Ao aprender apenas c, há 1306 vagas viáveis


## Modificação no algoritmo

In [14]:
def recommend_next_skill_modified(C, J, S, community_id):
    """
    Recommends the next skill to acquire based on current skills, job requirements, and a specified community.
    Args:
        C (list): A binary list indicating known (1) and unknown (0) skills.
        J (list): List of job identifiers.
        S (list): List of skill identifiers.
        community_id (int): The community identifier to filter skills.
    Returns:
        str or None: The recommended next skill to acquire, or None if no suitable skill is found.
    """
    T_J = []  # total de conhecimentos técnicos necessários para cada vaga de emprego
    C_J = []  # conhecimentos técnicos já conhecidos para cada vaga de emprego
    for job_index in range(len(J)):
        T_Ji = 0
        C_Ji = 0
        for skill_index in range(len(S)):
            if M[job_index][skill_index] == 1:
                T_Ji += 1
                if C[skill_index] == 1:
                    C_Ji += 1
        # Armazenando os resultados para uso futuro
        C_J.append(C_Ji)
        T_J.append(T_Ji)

    next_skill = None
    maximum_jobs = -1
    maximum_success = -1

    for skill_k_index, skill_k in enumerate(S):
        # Considerando apenas os conhecimentos técnicos ainda não adquiridos
        if C[skill_k_index] == 1:
            continue
        # Modificação: considerar apenas os conhecimentos técnicos pertencentes à comunidade especificada
        if skill_communities.nodes[skill_k]["community"] != community_id:
            continue

        jobs_k = 0
        success_k = 0

        for job_index in range(len(J)):
            known_skills = C_J[job_index]
            if M[job_index][skill_k_index] == 1:
                known_skills += 1

            percentage_k = known_skills / T_J[job_index]
            if percentage_k >= 0.5:
                jobs_k += 1
                success_k += percentage_k

        if jobs_k > maximum_jobs:
            maximum_jobs = jobs_k
            maximum_success = success_k
            next_skill = skill_k
        elif jobs_k == maximum_jobs and success_k > maximum_success:
            maximum_success = success_k
            next_skill = skill_k

    return next_skill

In [15]:
# Executando novamente a recomendação por comunidade com a modificação no algoritmo
for community_id, central_skills in central_skills_per_community.items():
    community_skills_C = create_C(S, central_skills)

    next_community_skill = recommend_next_skill_modified(
        community_skills_C, J, S, community_id
    )

    print(f"Comunidade {community_id}:")
    before_learning = get_viable_jobs_count(community_skills_C, J, S)
    print("  Vagas de emprego viáveis antes de aprender:", before_learning)
    print("  Conhecimento técnico recomendado:", next_community_skill)

    community_skills_C[S.index(next_community_skill)] = (
        1  # Atualizando C após aprender o conhecimento técnico recomendado
    )
    after_learning = get_viable_jobs_count(community_skills_C, J, S)
    print(
        f"  Vagas de emprego viáveis após aprender: {after_learning} (+{after_learning - before_learning})"
    )

Comunidade 1:
  Vagas de emprego viáveis antes de aprender: 1720
  Conhecimento técnico recomendado: css
  Vagas de emprego viáveis após aprender: 2473 (+753)
Comunidade 2:
  Vagas de emprego viáveis antes de aprender: 2035
  Conhecimento técnico recomendado: c
  Vagas de emprego viáveis após aprender: 3410 (+1375)
Comunidade 3:
  Vagas de emprego viáveis antes de aprender: 4628
  Conhecimento técnico recomendado: power bi
  Vagas de emprego viáveis após aprender: 7032 (+2404)
Comunidade 4:
  Vagas de emprego viáveis antes de aprender: 2657
  Conhecimento técnico recomendado: facebook
  Vagas de emprego viáveis após aprender: 3660 (+1003)
Comunidade 5:
  Vagas de emprego viáveis antes de aprender: 2178
  Conhecimento técnico recomendado: jira
  Vagas de emprego viáveis após aprender: 2920 (+742)
